In [ ]:
# app.py
from flask import Flask, render_template, request, jsonify, url_for
import os, time, logging, feedparser, requests, threading
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv
from flask_cors import CORS
from transformers import T5Tokenizer, T5ForConditionalGeneration
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup

# ----------------------- Bootstrap -----------------------
load_dotenv()
app = Flask(__name__)
CORS(app)
logging.basicConfig(level=logging.INFO)

@app.context_processor
def override_url_for():
    def dated_url_for(endpoint, **values):
        if endpoint == 'static':
            values['q'] = int(time.time())
        return url_for(endpoint, **values)
    return dict(url_for=dated_url_for)

# ----------------------- Globals -------------------------
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
analyzer = SentimentIntensityAnalyzer()

# Faster HTTP
SESSION = requests.Session()
SESSION.headers.update({
    "User-Agent": "Mozilla/5.0 (News-Aggregator/1.0; +https://example.com)"
})
REQUEST_TIMEOUT = 6

# Load T5 once
print("⏳ Loading T5 (small)…")
_t5_lock = threading.Lock()
with _t5_lock:
    T5_MODEL = T5ForConditionalGeneration.from_pretrained("t5-small")
    T5_TOKEN = T5Tokenizer.from_pretrained("t5-small")
print("✅ T5 loaded")

# ----------------------- Tiny TTL Cache -------------------
CACHE = {}
def set_cache(key, value, ttl=300):
    CACHE[key] = (time.time() + ttl, value)

def get_cache(key):
    v = CACHE.get(key)
    if not v: return None
    exp, val = v
    if time.time() > exp:
        CACHE.pop(key, None)
        return None
    return val

IMG_TTL = 60 * 60

# ----------------------- Utils ---------------------------
def get_sentiment_label(text: str) -> str:
    if not text: return "neutral"
    s = analyzer.polarity_scores(text).get("compound", 0)
    return "positive" if s > 0.05 else "negative" if s < -0.05 else "neutral"

def summarize_t5(text: str) -> str:
    if not text: return "No summary available."
    try:
        inp = "summarize: " + text.strip()
        inputs = T5_TOKEN.encode(inp, return_tensors="pt", max_length=512, truncation=True)
        ids = T5_MODEL.generate(
            inputs,
            max_length=42, min_length=12,
            length_penalty=2.0, num_beams=4, early_stopping=True
        )
        return T5_TOKEN.decode(ids[0], skip_special_tokens=True)
    except Exception as e:
        logging.warning(f"T5 summarize fail: {e}")
        return text[:200]

def _og_image_from_html(html, base_url):
    try:
        soup = BeautifulSoup(html, "html.parser")
        og = soup.find("meta", property="og:image") or soup.find("meta", attrs={"name": "og:image"})
        if og and og.get("content"): return og["content"]
        tw = soup.find("meta", attrs={"name": "twitter:image"}) or soup.find("meta", property="twitter:image")
        if tw and tw.get("content"): return tw["content"]
    except Exception:
        pass
    return None

def fetch_thumbnail(url: str):
    default_img = "https://via.placeholder.com/400x200?text=No+Image"  # Default placeholder
    if not url:
        return default_img
    cache_key = f"img:{url}"
    cached = get_cache(cache_key)
    if cached is not None:
        return cached or default_img
    try:
        r = SESSION.get(url, timeout=REQUEST_TIMEOUT)
        if r.ok:
            img = _og_image_from_html(r.text, url)
            if img:
                set_cache(cache_key, img, ttl=IMG_TTL)
                return img
    except Exception:
        pass
    # Cache None to prevent repeated failed requests, return default
    set_cache(cache_key, None, ttl=IMG_TTL//2)
    return default_img
    
def gl_ceid_for_lang(lang: str):
    lang = (lang or "en").lower()
    mapping = {
        "en": ("en", "IN", "IN:en"),
        "hi": ("hi", "IN", "IN:hi"),
        "or": ("or", "IN", "IN:or"),
        "es": ("es", "US", "US:es"),
        "fr": ("fr", "FR", "FR:fr"),
        "de": ("de", "DE", "DE:de"),
        "ja": ("ja", "JP", "JP:ja"),
    }
    return mapping.get(lang, ("en", "IN", "IN:en"))

def build_feed_url(query: str, lang: str):
    hl, gl, ceid = gl_ceid_for_lang(lang)
    q = requests.utils.quote(query)
    return f"https://news.google.com/rss/search?q={q}&hl={hl}&gl={gl}&ceid={ceid}"

# ----------------------- Multi-source Fetch ----------------------
def fetch_google_news(query, lang):
    url = build_feed_url(query, lang)
    feed = feedparser.parse(url)
    return [{
        "title": e.get("title", ""),
        "description": e.get("summary", ""),
        "url": e.get("link", ""),
        "publishedAt": e.get("published", ""),
        "source": "Google News"
    } for e in feed.entries]

def fetch_bing_news(query):
    url = f"https://www.bing.com/news/search?q={requests.utils.quote(query)}&format=rss"
    feed = feedparser.parse(url)
    return [{
        "title": e.get("title", ""),
        "description": e.get("summary", ""),
        "url": e.get("link", ""),
        "publishedAt": e.get("published", ""),
        "source": "Bing News"
    } for e in feed.entries]

def fetch_duck_news(query):
    url = f"https://duckduckgo.com/?q={requests.utils.quote(query)}&format=rss"
    feed = feedparser.parse(url)
    return [{
        "title": e.get("title", ""),
        "description": e.get("summary", ""),
        "url": e.get("link", ""),
        "publishedAt": e.get("published", ""),
        "source": "DuckDuckGo News"
    } for e in feed.entries]

# ----------------------- Fast Fetcher ----------------------
def fetch_news_fast(query="technology", language="en", page=1, page_size=8):
    cache_key = f"multi:{query}:{language}"
    feed_items = get_cache(cache_key)
    if feed_items is None:
        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = [
                executor.submit(fetch_google_news, query, language),
                executor.submit(fetch_bing_news, query),
                executor.submit(fetch_duck_news, query)
            ]
            feed_items = []
            for f in as_completed(futures):
                try:
                    feed_items.extend(f.result())
                except Exception:
                    pass
        # Cache results for 5 minutes
        set_cache(cache_key, feed_items, ttl=300)

    total = len(feed_items)
    start = max((page - 1) * page_size, 0)
    end = min(start + page_size, total)
    slice_items = feed_items[start:end]

    def enrich(item):
        title = item["title"]
        desc = item["description"]
        text = (desc or title)[:800]
        summary = summarize_t5(text)
        img = fetch_thumbnail(item["url"])
        sent = get_sentiment_label(f"{title}. {summary}")
        item.update({
            "summary": summary,
            "image": img,
            "sentiment": sent
        })
        return item

    results = []
    with ThreadPoolExecutor(max_workers=6) as ex:
        futures = [ex.submit(enrich, dict(it)) for it in slice_items]
        for f in as_completed(futures):
            try:
                results.append(f.result()) 
            except:
                pass

    url_to_idx = {item["url"]: i for i, item in enumerate(slice_items)}
    results.sort(key=lambda it: url_to_idx.get(it["url"], 0))

    has_more = end < total
    return results, has_more

# ----------------------- Routes -------------------------
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get_news", methods=["GET"])
def get_news():
    q = request.args.get("query", "technology")
    lang = request.args.get("language", "en")
    page = int(request.args.get("page", "1") or "1")
    page_size = int(request.args.get("page_size", "8") or "8")

    logging.info(f"[NEWS] q='{q}' lang={lang} page={page}")
    try:
        items, has_more = fetch_news_fast(q, lang, page, page_size)
        return jsonify({
            "error": False,
            "items": items,
            "source": "Google + Bing + DuckDuckGo + T5",
            "page": page,
            "page_size": page_size,
            "has_more": has_more
        })
    except Exception as e:
        logging.exception("get_news failed")
        return jsonify({"error": True, "message": str(e), "items": [], "source": "Multi News", "has_more": False})

@app.route("/health")
def health():
    return jsonify({"ok": True})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)

⏳ Loading T5 (small)…


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


✅ T5 loaded
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.44.192.26:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.44.192.26 - - [09/Nov/2025 01:42:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.44.192.26 - - [09/Nov/2025 01:42:54] "GET /static/style.css?q=1762632774 HTTP/1.1" 200 -
INFO:werkzeug:10.44.192.26 - - [09/Nov/2025 01:42:54] "GET /static/script.js?q=1762632774 HTTP/1.1" 200 -
INFO:root:[NEWS] q='technology' lang=en page=1
INFO:werkzeug:10.44.192.26 - - [09/Nov/2025 01:43:30] "GET /get_news?query=technology&language=en&page=1&page_size=6 HTTP/1.1" 200 -
INFO:root:[NEWS] q='science' lang=en page=1
INFO:werkzeug:10.44.192.26 - - [09/Nov/2025 01:44:26] "GET /get_news?query=science&language=en&page=1&page_size=6 HTTP/1.1" 200 -


In [ ]:
pip install flask==2.3.3 werkzeug==2.3.7

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall werkzeug -y

Found existing installation: Werkzeug 2.3.7
Uninstalling Werkzeug-2.3.7:
  Successfully uninstalled Werkzeug-2.3.7
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install werkzeug==2.3.7

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade flask flask-cors werkzeug==2.3.7

INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip show flask werkzeug flask-cors

Name: Flask
Version: 2.3.3
Summary: A simple framework for building complex web applications.
Home-page: 
Author: 
Author-email: 
License: 
Location: c:\users\nr143\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
Requires: blinker, click, itsdangerous, Jinja2, Werkzeug
Required-by: flask-cors
---
Name: Werkzeug
Version: 2.3.7
Summary: The comprehensive WSGI web application library.
Home-page: 
Author: 
Author-email: 
License: 
Location: c:\users\nr143\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
Requires: MarkupSafe
Required-by: Flask, flask-cors, tensorboard
---
Name: flask-cors
Version: 6.0.1
Summary: A Flask extension simplifying CORS support
Home-page: https://corydolphin.github.io/flask-cors/
Author: 
Author-email: Cory Dolphin <corydolphin@gmail.com>
License: 
Location: c:\users\nr143\appdata\local\packages\pythonsoftwarefounda

In [ ]:
pip cache purge

Files removed: 20 (88 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sentencepiece --upgrade


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sentencepiece
print("SentencePiece installed successfully!")


SentencePiece installed successfully!


In [ ]:
pip install transformers torch newspaper3k


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip show python-dotenv


Name: python-dotenv
Version: 1.1.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: https://github.com/theskumar/python-dotenv
Author: Saurabh Kumar
Author-email: me+github@saurabh-kumar.com
License: BSD-3-Clause
Location: C:\Users\nr143\AppData\Roaming\Python\Python313\site-packages
Requires: 
Required-by: anaconda-auth, pydantic-settings
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pipeline

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -r requirements.txt


  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)

The conflict is caused by:
    The user requested Flask==3.0.3
    The user requested flask==2.3.2

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

Note: you may need to restart the kernel to use updated packages.


ERROR: Cannot install Flask==3.0.3 and flask==2.3.2 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
pip install vaderSentiment


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers torch sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install feedparser

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install flask-cors


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install flask-cors

In [ ]:
pip show vaderSentiment

Name: vaderSentiment
Version: 3.3.2
Summary: VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.
Home-page: https://github.com/cjhutto/vaderSentiment
Author: C.J. Hutto
Author-email: cjhutto@gatech.edu
License: MIT License
Location: C:\Users\nr143\AppData\Roaming\Python\Python313\site-packages
Requires: requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.
